# **OpenVisus Tutorial: Terrain Parameters**



This tutorial uses the following architecture:

<p align="center">
    <img src="files/docs/Openvisus-somospie.png" width="800">
</p>

### Import Dependencies
Run this cell to import all the dependencies you need to run the entire notebook

In [ ]:
import geotiled as gt
from pathlib import Path
import glob
import os
import shutil
import multiprocessing
import OpenVisus as ov
import numpy as np
import requests
import json
from matplotlib import pyplot as plt
from tqdm import tqdm

# To silence a deprecation warning.
gt.gdal.UseExceptions()

## Step 1 - Data Generation

The first step talks about the data source for this particular example, we are using the [SOMOSPIE](https://github.com/TauferLab/SOMOSPIE) workflow with which we will generate the terrain parameters for the state of Tennessee, the files generated by this workflow are images in Tiff format.


### Option 1. Generating data scratch

Run these cells in case you want to generate the terrain parameters from scratch

> **Note:** All the necessary files are already inside the container. 

In [ ]:
download_list = "./download_urls.txt"  # Where the list of download links will be stored
root_output_folder = "./files/tif_files/"  # root folder where geotiled will store data
n_tiles = 4  # Number of tiles that are generated for parameter computation
dem_tiles_dir_name = "tiles"  # Folder where downloaded DEM tiles will be saved
param_tiles_dir_name = (
    "elevation_tiles"  # Folder where computation tiles will be saved.
)
gcs_name = "gcs.tif"  # Name for the mosaicked DEM
pcs_name = "pcs.tif"  # Name for the projected DEM
shapefile = ["./files/shape_files/STATEFP_47.shp"]  # Shapefile for Visualization
region_bounding_box = {
    "xmin": -90.4,
    "ymin": 34.8,
    "xmax": -81.55,
    "ymax": 36.8,
}  # For `fetch_dem`. X=Longitude Y=Latitude. Determine bounding coordinates by looking at a map.

In [ ]:
# Fetching Data
tiles_folder = os.path.join(root_output_folder, dem_tiles_dir_name)
Path(root_output_folder).mkdir(parents=True, exist_ok=True)
Path(tiles_folder).mkdir(parents=True, exist_ok=True)

# Setting up for parameter computation
gcs = os.path.join(root_output_folder, gcs_name)
pcs = os.path.join(root_output_folder, pcs_name)
elevation_tiles = os.path.join(root_output_folder, param_tiles_dir_name)
Path(elevation_tiles).mkdir(parents=True, exist_ok=True)

# Computing Parameters
aspect_tiles = os.path.join(root_output_folder, "aspect_tiles")
hillshading_tiles = os.path.join(root_output_folder, "hillshading_tiles")
slope_tiles = os.path.join(root_output_folder, "slope_tiles")
Path(aspect_tiles).mkdir(parents=True, exist_ok=True)
Path(hillshading_tiles).mkdir(parents=True, exist_ok=True)
Path(slope_tiles).mkdir(parents=True, exist_ok=True)

In [ ]:
gt.fetch_dem(
    bbox=region_bounding_box,
    txtPath=download_list,
    dataset="National Elevation Dataset (NED) 1 arc-second Current",
)
gt.download_files(download_list, tiles_folder)

In [ ]:
raster_list = glob.glob(tiles_folder + "/*")

gt.build_mosaic(raster_list, gcs)

shutil.rmtree(tiles_folder)
os.remove("./merged.vrt")

In [ ]:
gt.reproject(gcs, pcs, "EPSG:9822")

os.remove(gcs)

In [ ]:
gt.crop_into_tiles(pcs, elevation_tiles, n_tiles)

glob_of_tiles = glob.glob(elevation_tiles + "/*.tif")

In [ ]:
pool = multiprocessing.Pool(processes=n_tiles)
pool.map(gt.compute_geotiled, sorted(glob.glob(elevation_tiles + "/*.tif")))

In [ ]:
gt.build_mosaic_filtered(
    sorted(glob.glob(aspect_tiles + "/*.tif")),
    os.path.join(root_output_folder, "aspect.tif"),
)
gt.build_mosaic_filtered(
    sorted(glob.glob(hillshading_tiles + "/*.tif")),
    os.path.join(root_output_folder, "hillshading.tif"),
)
gt.build_mosaic_filtered(
    sorted(glob.glob(slope_tiles + "/*.tif")),
    os.path.join(root_output_folder, "slope.tif"),
)


shutil.rmtree(aspect_tiles)
shutil.rmtree(hillshading_tiles)
shutil.rmtree(slope_tiles)
shutil.rmtree(elevation_tiles)

In [ ]:
hill = os.path.join(root_output_folder, "hillshading.tif")
aspect = os.path.join(root_output_folder, "aspect.tif")
slope = os.path.join(root_output_folder, "slope.tif")


pcs_array = gt.generate_img(
    pcs,
    downsample=5,
    reproject_gcs=True,
    shp_files=shapefile,
    title="Elevation Data for TN @ 1 Arc-Second/30m Resolution",
    zunit="Meter",
    xyunit="Degree",
    ztype="Elevation",
    crop_shp=True,
)
hill_array = gt.generate_img(
    hill,
    downsample=5,
    reproject_gcs=True,
    shp_files=shapefile,
    title="Hillshading Data for TN @ 1 Arc-Second/30m Resolution",
    zunit="Level",
    xyunit="Degree",
    ztype="Hillshading",
    crop_shp=True,
)
aspect_array = gt.generate_img(
    aspect,
    downsample=5,
    reproject_gcs=True,
    shp_files=shapefile,
    title="Aspect Data for Rhode TN @ 1 Arc-Second/30m Resolution",
    zunit="Degree",
    xyunit="Degree",
    ztype="Aspect",
    crop_shp=True,
)
slope_array = gt.generate_img(
    slope,
    downsample=5,
    reproject_gcs=True,
    shp_files=shapefile,
    title="Slope Data for TN @ 1 Arc-Second/30m Resolution",
    zunit="Degree",
    xyunit="Degree",
    ztype="Slope",
    crop_shp=True,
)

### Option 2. Fetching data from dataverse

Run these cells in case you want to download the previously generated tiff files from dataverse

In [ ]:
if not os.path.exists("files/tif_files"):
    os.mkdir("files/tif_files")

with open("./files/json/dataverse.json", "r") as file:
    urls = json.load(file)


def get_data_from_dataverse(file_url, name_file):
    resp = requests.get(file_url)
    with open(name_file, "wb") as f:
        f.write(resp.content)


for data in tqdm(urls):
    get_data_from_dataverse(data.get("url"), data.get("name_file"))

In [ ]:
shapefile = ["./files/shape_files/STATEFP_47.shp"]  # Shapefile for Visualization
hill = os.path.join("./files/tif_files/", "TN_30M_hillshading.tif")
pcs = os.path.join("./files/tif_files/", "TN_30M_aspect.tif")
aspect = os.path.join("./files/tif_files/", "TN_30M_elevation.tif")
slope = os.path.join("./files/tif_files/", "TN_30M_slope.tif")
pcs_array = gt.generate_img(
    pcs,
    downsample=5,
    reproject_gcs=True,
    shp_files=shapefile,
    title="Elevation Data for TN @ 1 Arc-Second/30m Resolution",
    zunit="Meter",
    xyunit="Degree",
    ztype="Elevation",
    crop_shp=True,
)
hill_array = gt.generate_img(
    hill,
    downsample=5,
    reproject_gcs=True,
    shp_files=shapefile,
    title="Hillshading Data for TN @ 1 Arc-Second/30m Resolution",
    zunit="Level",
    xyunit="Degree",
    ztype="Hillshading",
    crop_shp=True,
)
aspect_array = gt.generate_img(
    aspect,
    downsample=5,
    reproject_gcs=True,
    shp_files=shapefile,
    title="Aspect Data for Rhode TN @ 1 Arc-Second/30m Resolution",
    zunit="Degree",
    xyunit="Degree",
    ztype="Aspect",
    crop_shp=True,
)
slope_array = gt.generate_img(
    slope,
    downsample=5,
    reproject_gcs=True,
    shp_files=shapefile,
    title="Slope Data for TN @ 1 Arc-Second/30m Resolution",
    zunit="Degree",
    xyunit="Degree",
    ztype="Slope",
    crop_shp=True,
)

## Step 2 - Conversion to IDX
Once the data is generated, it must be converted into the IDX format so that it can be read by OpenVisus. 
Here, we are only creating one single idx file, and including all these variables as fields.

Now, for all fields of idx, we are writing the corresponding data. `Fields` should be in this format: [ov.Field(FIELD_NAME, DTYPE)]


After writing, we are compressing this with `zip` compression.

**Note:** To execute this cell it is mandatory to first complete one of the options in step 1 mentioned above.

In [ ]:
filename = "idx_data/Tennessee_terrain_parameters.idx"
all_fields = [
    ov.Field("elevation", "float32"),
    ov.Field("hillshading", "uint8"),
    ov.Field("aspect", "float32"),
    ov.Field("slope", "float32"),
]
input_data = [
    np.flipud(pcs_array).copy(),
    np.flipud(hill_array).copy(),
    np.flipud(aspect_array).copy(),
    np.flipud(slope_array).copy(),
]
height, width = input_data[0].shape
db = ov.CreateIdx(
    url=filename,
    dims=[width, height],
    fields=all_fields,
    arco="4mb",
    time=[0, 0, "%00000d/"],
)
i = 0
for fld in db.getFields():
    db.write(input_data[i], field=fld)
    i += 1
db.compressDataset(["zip"])

## Step 3 - Visualization

### Option 1. Load dataset from a local file
Run this cell to load dataset from a local file

In [ ]:
filename = "idx_data/Tennessee_terrain_parameters.idx"
db = ov.LoadDataset(filename)

In [ ]:
read_elevation = db.read(field="elevation")
read_hillshading = db.read(field="hillshading")
read_aspect = db.read(field="aspect")
read_slope = db.read(field="slope")

In [ ]:
fig, axs = plt.subplots(4, 1, figsize=(10, 8))
axs[0].imshow(read_elevation, origin="lower", vmin=30, vmax=1999, cmap="BuPu_r")
axs[0].set_title("Elevation")
axs[1].imshow(read_hillshading, origin="lower", vmin=0, vmax=255, cmap="Oranges")
axs[1].set_title("Hillshading")

axs[2].imshow(read_aspect, vmin=0, origin="lower", vmax=360, cmap="Reds")
axs[2].set_title("Aspect")

axs[3].imshow(read_slope, vmin=0, origin="lower", vmax=65.9, cmap="Oranges")
axs[3].set_title("Slope")
plt.subplots_adjust(wspace=0.4, hspace=0.6)
plt.tight_layout()
plt.show()

### Option 2. Load dataset from Seal Storage
Run this cell to load dataset from Seal Storage

In [ ]:
filename = "Tennessee_terrain_parameters.idx"
HOME_DIR = "s3://utah/nsdf/somospie/"  # DONOT change this line
data_dir = "terrain_tennessee/"
upload_dir = HOME_DIR + data_dir
s3_path = upload_dir.split("://")[1]
s3_path += filename
remote_dir = (
    "https://maritime.sealstorage.io/api/v0/s3/"
    + s3_path
    + "?access_key=any&secret_key=any&endpoint_url=https://maritime.sealstorage.io/api/v0/s3&cached=arco"
)

db = ov.LoadDataset(remote_dir)

In [ ]:
read_elevation = db.read(field="elevation")
read_hillshading = db.read(field="hillshading")
read_aspect = db.read(field="aspect")
read_slope = db.read(field="slope")

In [ ]:
fig, axs = plt.subplots(4, 1, figsize=(10, 8))
axs[0].imshow(read_elevation, origin="lower", vmin=30, vmax=1999, cmap="BuPu_r")
axs[0].set_title("Elevation")
axs[1].imshow(read_hillshading, origin="lower", vmin=0, vmax=255, cmap="Oranges")
axs[1].set_title("Hillshading")

axs[2].imshow(read_aspect, vmin=0, origin="lower", vmax=360, cmap="Reds")
axs[2].set_title("Aspect")

axs[3].imshow(read_slope, vmin=0, origin="lower", vmax=65.9, cmap="Oranges")
axs[3].set_title("Slope")
plt.subplots_adjust(wspace=0.4, hspace=0.6)
plt.tight_layout()
plt.show()

## Create a dashboard using OpenVisus

### Option 1. Load dataset from Seal Storage
Run this cell to create the dashboard collecting data from Seal Storage

In [ ]:
!python3 -m bokeh serve --port 8989 dashboard.py

### Option 2. Load dataset from a local file
Run this cell to create the dashboard collecting data from a local file

In [ ]:
!python3 -m bokeh serve --port 8989 dashboard.py --remote False